# Ajuste do SUOD
Teste do SUOD  para comparação e seleção dos integrantes do aplicativo

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV
from pyod.utils.data import evaluate_print
import matplotlib.pyplot as plt

tabelafinal = pd.DataFrame(columns=['algoritmo','contaminacao','acuracia','precisao','recall','f1','fn','fp','tp'])
print(tabelafinal)
 
def avalia(clf_name,Y,y_train_scores,y_train_pred):
    # Avalia e imprime os resultados
    print("Avaliação do modelo "+clf_name+":")
    evaluate_print(clf_name, Y, y_train_scores)
    tn, fp, fn, tp = confusion_matrix(Y,y_train_pred).ravel()
    print('Acurácia='+str(accuracy_score(Y,y_train_pred)*100))   
    print('Precisão='+str(precision_score(Y,y_train_pred)*100))
    print('Recall='+str(recall_score(Y,y_train_pred)*100))
    print('Falso Positivo='+str(fp))
    print('Falso Negativo='+str(fn))
    print('F1-score='+str(f1_score(Y,y_train_pred)*100))
    print('ROC='+str(roc_auc_score(Y,y_train_scores)))

def gera_matriz_de_confusao(Y,y_train_pred,clf_name):
    cm = confusion_matrix(Y,y_train_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal','Anomalia'])
    disp.plot(cmap=plt.cm.Blues)
    disp.ax_.set(title='Matriz de Confusão do '+clf_name,xlabel='Valores preditos', ylabel='Valores reais')
    plt.show()
    
def grava_avaliacao(clf_name,Y,y_train_scores,y_train_pred,contamination):
    # Grava os resultados da avaliação na tabela final
    tn, fp, fn, tp = confusion_matrix(Y,y_train_pred).ravel()
    ac=("{:.2f}".format(round(accuracy_score(Y,y_train_pred)*100,2))).replace(".", ",")
    print('Acurácia='+ac)   
    pc=("{:.2f}".format(round(precision_score(Y,y_train_pred)*100,2))).replace(".", ",")
    print('Precisão='+pc)
    rc=("{:.2f}".format(round(recall_score(Y,y_train_pred)*100,2))).replace(".", ",")
    print('Recall='+rc)
    f1=("{:.2f}".format(round(f1_score(Y,y_train_pred)*100,2))).replace(".", ",")
    print('F1='+f1)
    print('Falso Negativo='+str(fn))
    print('Falso Positivo='+str(fp))
    print('Verdadeiro Positivo='+str(tp))
    cont=str(round(contamination,2)).replace(".",",")
    tabelafinal.loc[len(tabelafinal.index)] = [clf_name,cont,ac,pc,rc,f1,fn,fp,tp] 
      

Empty DataFrame
Columns: [algoritmo, contaminacao, acuracia, precisao, recall, f1, fn, fp, tp]
Index: []


# Recuperação dos dados a serem utilizados na avaliação
Leitura do arquivo gerado para o material de catmat_id= 469793 -> Descrição:'MICROCOMPUTADOR'.

In [2]:
import os
import pandas as pd
def load_itens(path):
    csv_path = os.path.join(path,"469793.csv")
    return pd.read_csv(csv_path)

df=load_itens(".")
## Retira os campos que não serão utilizados no treinamento
df = df.drop(['licitacao_contrato'],axis=1)
df = df.drop(['id'],axis=1)
df = df.drop(['data'],axis=1)
df = df.drop(['catmat_id'],axis=1)
df = df.drop(['unidade'],axis=1)
df = df.drop(['valor_total'],axis=1)
df = df.drop(['municipio_uasg'],axis=1)
df = df.drop(['municipio_fornecedor'],axis=1)
print(df)

     quantidade  valor_unitario  distancia_uasg_fornecedor  anomalia
0           100          6570.0                   2346.440         1
1             0             0.0                      0.000         0
2             0             0.0                      0.000         0
3             0             0.0                      0.000         0
4             0             0.0                      0.000         0
..          ...             ...                        ...       ...
272           0             0.0                      0.000         0
273           0             0.0                      0.000         0
274           0             0.0                      0.000         0
275           4         45000.0                      0.000         1
276          65          8862.0                    531.952         1

[277 rows x 4 columns]


# Separação do Label dos dados

In [3]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
print(x)
print(y)

     quantidade  valor_unitario  distancia_uasg_fornecedor
0           100          6570.0                   2346.440
1             0             0.0                      0.000
2             0             0.0                      0.000
3             0             0.0                      0.000
4             0             0.0                      0.000
..          ...             ...                        ...
272           0             0.0                      0.000
273           0             0.0                      0.000
274           0             0.0                      0.000
275           4         45000.0                      0.000
276          65          8862.0                    531.952

[277 rows x 3 columns]
0      1
1      0
2      0
3      0
4      0
      ..
272    0
273    0
274    0
275    1
276    1
Name: anomalia, Length: 277, dtype: int64


# SUOD

In [ ]:
# Importa bibliotecas do PyOD com os algoritmos de detecção de anomalias
from pyod.models.inne import INNE
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.pca import PCA
from pyod.models.sampling import Sampling
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.suod import SUOD
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score


detector_list = [
         INNE(contamination=0.05, n_estimators=50, random_state= 69), 
         ABOD(contamination=0.05, method= 'default', n_neighbors=20),
         CBLOF(contamination=0.05, n_clusters=2),
         KNN(contamination=0.05, leaf_size=10, method='largest', n_neighbors=10),
         LOF(contamination=0.05, leaf_size=1, n_neighbors=34),  
         PCA(contamination=0.05, n_components=3, n_selected_components=1),
         Sampling(contamination=0.05, subset_size=10),
         ECOD(contamination=0.09),
         COPOD(contamination=0.12)
]
    
# Inicializar o modelo SUOD
model = SUOD(base_estimators=detector_list, n_jobs=2, combination='maximization',rp_flag_global=False,approx_flag_global=False)

# Definir a validação cruzada estratificada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=69)


acuracia=[]
recall=[]

# Loop para treinar e testar o modelo
for train_index, test_index in cv.split(x,y):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Treinar o modelo
    model.fit(X_train)

    # Fazer as previsões
    y_pred = model.predict(X_test)

    # Calcular a acurácia
    acc = accuracy_score(y_test, y_pred)
    print(f'Acurácia: {acc}')
    acuracia.append(acc)
    
    # Calcular o Recall
    rec = recall_score(y_test, y_pred)
    print(f'Recall: {rec}')
    recall.append(rec)
    
acuracia_media = sum(acuracia) / len(acuracia)
recall_medio = sum(recall) / len(recall)
    
    
print(f'Acurácia média: {acuracia_media}')
print(f'Recall médio: {recall_medio}')



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.4min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s finished


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   10.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   10.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Acurácia: 1.0
Recall: 1.0



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.5s finished


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


...

Acurácia: 1.0
Recall: 1.0



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s finished


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   21.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   21.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Acurácia: 1.0
Recall: 1.0



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.7min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


# Imprime a tabela final de avaliação

In [ ]:
tabelafinal.sort_values("fn", axis = 0, ascending = True,
                 inplace = True, na_position ='last')
print(tabelafinal)